In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Multiple_Chemical_Sensitivity/GSE182503'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic profiling of adult-onset asthma related to damp and moldy buildings and idiopathic environmental intolerance [blood]"
!Series_summary	"The objective of the study was to characterize distinct endotypes of asthma related to damp and moldy buildings and to evaluate the potential molecular similarities with idiopathic environmental intolerance (IEI). The blood cell transcriptome of 88 study subjects was profiled using peripheral blood mononuclear cell (PBMC) samples obtained at baseline."
!Series_overall_design	"Venous blood samples were collected from female adult-onset asthma patients (n=50), IEI patients (n=17) and healthy subjects (n=21) yielding 88 study subjects. PBMCs were separated and total RNA extracted for microarray analyses."
Sample Characteristics Dictionary:
{0: ['diagnosis: adult-onset asthma', 'diagnosis: IEI', 'diagnosis: healthy'], 1: ['gender: Female'], 2: ['age: 33.42', 'age: 46.08', 'age: 45.58', 'age: 28', 'age: 2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # This dataset contains gene expression data from PBMCs

# Data availability
trait_row = 0  # Using 'diagnosis' as a proxy for 'Multiple_Chemical_Sensitivity'
age_row = 2  # Age data is available under key 2
gender_row = None  # Gender data only has one unique value (Female), hence not useful

# Data type conversions
def convert_trait(value):
    val = value.split(':')[-1].strip().lower()
    if "healthy" in val:
        return 0
    elif "asthma" in val or "iei" in val:
        return 1
    else:
        return None

def convert_age(value):
    try:
        # Extract value after the colon and convert to float
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None  # Return None if the conversion fails

save_cohort_info('GSE182503', './preprocessed/Multiple_Chemical_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)

# since trait_row is not None, perform clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Multiple_Chemical_Sensitivity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Multiple_Chemical_Sensitivity/trait_data/GSE182503.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5530417': [1.0, 33.42], 'GSM5530418': [1.0, 46.08], 'GSM5530419': [1.0, 45.58], 'GSM5530420': [0.0, 28.0], 'GSM5530421': [0.0, 25.75], 'GSM5530422': [1.0, 59.83], 'GSM5530423': [0.0, 41.17], 'GSM5530424': [1.0, 47.58], 'GSM5530425': [1.0, 50.75], 'GSM5530426': [0.0, 42.58], 'GSM5530427': [1.0, 52.75], 'GSM5530428': [1.0, 51.75], 'GSM5530429': [0.0, 18.42], 'GSM5530430': [0.0, 47.0], 'GSM5530431': [0.0, 38.33], 'GSM5530432': [1.0, 58.58], 'GSM5530433': [1.0, 56.17], 'GSM5530434': [1.0, 52.75], 'GSM5530435': [1.0, 40.67], 'GSM5530436': [1.0, 47.5], 'GSM5530437': [1.0, 54.67], 'GSM5530438': [0.0, 48.83], 'GSM5530439': [0.0, 25.75], 'GSM5530440': [1.0, 64.67], 'GSM5530441': [1.0, 54.83], 'GSM5530442': [1.0, 57.67], 'GSM5530443': [1.0, 39.17], 'GSM5530444': [0.0, 38.08], 'GSM5530445': [1.0, 28.42], 'GSM5530446': [1.0, 40.75], 'GSM5530447': [1.0, 43.17], 'GSM5530448': [0.0, 43.08], 'GSM5530449': [1.0, 48.83], 'GSM5530450': [0.0, 58.83], 'GSM5530451': [0.0, 26.58], 'GSM5530452': [0.0, 42

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315506',
       'A_19_P00315529', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581',
       'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315627',
       'A_19_P00315649', 'A_19_P00315668', 'A_19_P00315691', 'A_19_P00315705',
       'A_19_P00315716', 'A_19_P00315753', 'A_19_P00315764', 'A_19_P00315773'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Multiple_Chemical_Sensitivity/gene_data/GSE182503.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Multiple_Chemical_Sensitivity')

# 4. Save the cohort information.
save_cohort_info('GSE182503', './preprocessed/Multiple_Chemical_Sensitivity/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Multiple_Chemical_Sensitivity/GSE182503.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Multiple_Chemical_Sensitivity', the least common label is '0.0' with 36 occurrences. This represents 35.29% of the dataset.
The distribution of the feature 'Multiple_Chemical_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 40.58
  50% (Median): 47.0
  75%: 52.75
Min: 18.42
Max: 64.67
The distribution of the feature 'Age' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Multiple_Chemical_Sensitivity/cohort_info.json
